#Modelos de Crédito

## Proyecto Final
##  Dataset: The Winton Stock Market Challenge
### Juan Antonio Flores Romero



![texto alternativo](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Logo-ITESO-Principal-60V.png/250px-Logo-ITESO-Principal-60V.png)



